In [9]:
#coding: utf-8
import datetime as dt
import pandas as pd
import numpy as np
import os

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont  
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

from reportlab.lib.units import inch
from reportlab.lib import fonts,colors
from reportlab.pdfbase.ttfonts import TTFont

In [10]:
cwd = os.getcwd()
cwd

'/Users/jingkun/Dropbox/DG/id-list-generator'

In [11]:
date_today = dt.date.today().strftime("%m-%d-%Y")
inputXLSX = date_today + "_shipping-list.xlsx"
outputPDF = date_today + "_usatocn2013_YU_身份证.pdf"

In [26]:
df_shipList = pd.read_excel(inputXLSX)
df_shipList = df_shipList[[u"制单时间",u"运单号",u"收件人"]]
n0 = len(df_shipList)
print "%s一共有%s个包裹"%(date_today,n0)
print df_shipList

09-13-2016一共有22个包裹
                  制单时间         运单号  收件人
0  2016-09-12 19:30:52  8000093495  胡磊菲
1  2016-09-12 19:26:29  8000093493  贾政晴
2  2016-09-12 19:21:07  8000093492  庄泾坤
3  2016-09-12 19:08:53  8000093491  冯云儿
4  2016-09-12 19:06:02  8000093490  余昕玲
5  2016-09-12 19:00:23  8000093488  刘松娟
6  2016-09-12 14:07:00  8000093468  高辰夏
7  2016-09-12 10:07:09  8000093427   曾艳
8  2016-09-12 10:03:56  8000093426  冯云儿
9  2016-09-12 10:02:25  8000093425   胡冰
10 2016-09-12 09:43:56  8000093424  丁霞呤
11 2016-09-12 09:42:32  8000093423  厉伟平
12 2016-09-12 19:23:32  8000093422   邵云
13 2016-09-12 09:37:35  8000093421  闵迎辉
14 2016-09-12 09:34:19  8000093419  王继民
15 2016-09-08 19:02:26  8000093200  吴燕茹
16 2016-09-08 18:33:56  8000093199  庄泾坤
17 2016-09-08 08:09:33  8000093192  余昕玲
18 2016-09-07 14:31:50  8000093115  潘清源
19 2016-09-07 13:25:01  8000093113  余万立
20 2016-09-07 13:24:11  8000093112  刘松娟
21 2016-09-07 11:03:08  8000093083  潘武华


In [27]:
#当同一收件人有多个包裹时，只保留第一个包裹单号
df_shipList = df_shipList.drop_duplicates(subset = u"收件人", keep = "first")
print "%s我们一共送了%s个包裹"%(date_today,n0)
print "当同一收件人有多个包裹时，身份证清单中只保留第一个包裹"
print "移除重名后，身份证清单中一共有%s个包裹"%(len(df_shipList))
print df_shipList

09-13-2016我们一共送了22个包裹
当同一收件人有多个包裹时，身份证清单中只保留第一个包裹
移除重名后，身份证清单中一共有18个包裹
                  制单时间         运单号  收件人
0  2016-09-12 19:30:52  8000093495  胡磊菲
1  2016-09-12 19:26:29  8000093493  贾政晴
2  2016-09-12 19:21:07  8000093492  庄泾坤
3  2016-09-12 19:08:53  8000093491  冯云儿
4  2016-09-12 19:06:02  8000093490  余昕玲
5  2016-09-12 19:00:23  8000093488  刘松娟
6  2016-09-12 14:07:00  8000093468  高辰夏
7  2016-09-12 10:07:09  8000093427   曾艳
9  2016-09-12 10:02:25  8000093425   胡冰
10 2016-09-12 09:43:56  8000093424  丁霞呤
11 2016-09-12 09:42:32  8000093423  厉伟平
12 2016-09-12 19:23:32  8000093422   邵云
13 2016-09-12 09:37:35  8000093421  闵迎辉
14 2016-09-12 09:34:19  8000093419  王继民
15 2016-09-08 19:02:26  8000093200  吴燕茹
18 2016-09-07 14:31:50  8000093115  潘清源
19 2016-09-07 13:25:01  8000093113  余万立
21 2016-09-07 11:03:08  8000093083  潘武华


In [25]:
#pdfmetrics.registerFont(TTFont("haha", "simsun.ttc"))

c3 = SimpleDocTemplate(outputPDF,pagesize=letter, rightMargin=72,leftMargin=72, topMargin=72,bottomMargin=18)
#c3.fontName   = "haha"
#c3.fontSize   = 12

idList = []
#missingID = []
missingN = 0

styles=getSampleStyleSheet()


for n in range(len(df_shipList)):
    # print tracking number
    tracking = str(df_shipList[n:n+1][u"运单号"].iloc[0])
    
    
    # print name
    #name = str(df_shipList[n:n+1]["name"].iloc[0].encode('utf-8'))
    #idList.append(Paragraph(u"刘女士",styles["Normal"]))
    
    # print photo-id
    try:
        id_0 = "./photo-id/" + df_shipList[n:n+1][u"收件人"].iloc[0].encode('utf-8') + ".jpg"
        im_0 = Image(id_0, 4*inch, 6*inch, kind='proportional')
        idList.append(Paragraph(tracking,styles["Normal"]))
        idList.append(im_0)
        idList.append(PageBreak())
    except IOError:
        try:
            id_1 = "./photo-id/" + df_shipList[n:n+1][u"收件人"].iloc[0].encode('utf-8') + "1.jpg"
            im_1 = Image(id_1, 4*inch, 4*inch, kind='proportional')
            im_1True = True
        except IOError:
#            missingID.append(id_1)
            im_1True = False
#            idList.append(Paragraph("Photo ID not available", styles["Normal"]))
            print id_1[11:-5] + '\t' + "正面" + ' ' + "身份证没有找到"
            missingN += 1
            
        if im_1True is True:
            idList.append(Paragraph(tracking,styles["Normal"]))
            idList.append(im_1)
            idList.append(Spacer(1,12))
            try:
                id_2 = "./photo-id/" + df_shipList[n:n+1][u"收件人"].iloc[0].encode('utf-8') + "2.jpg"
                im_2 = Image(id_2, 4*inch, 4*inch, kind='proportional')
                idList.append(im_2)
                im_2True = True
            except IOError:
                im_2True = False
                print id_1[11:-5] + '\t' + "背面" + ' ' + "身份证没有找到"
                missingN += 1
            idList.append(PageBreak())
print
print "一共%s个运单，其中%s个没有身份证"%(n+1,missingN)
print "生成文件已经保存到 " + cwd + " 目录下面"
print "文件名为 " + outputPDF
            
print
print "PS:"
print "请把缺失的身份证图片添加到" + cwd + "/photo-id 目录下面"
print "如果正反面在同一张图片上，命名为 姓名.jpg"
print "如果正反面分两张图片，命名为 姓名1.jpg 和 姓名2.jpg"

    

c3.build(idList)
#missingID

胡磊菲	正面 身份证没有找到
贾政晴	正面 身份证没有找到
高辰夏	正面 身份证没有找到
曾艳	正面 身份证没有找到
胡冰	正面 身份证没有找到
丁霞呤	正面 身份证没有找到
厉伟平	正面 身份证没有找到
邵云	正面 身份证没有找到
闵迎辉	正面 身份证没有找到
王继民	正面 身份证没有找到
余万立	背面 身份证没有找到

一共18个运单，其中11个没有身份证
生成文件已经保存到 /Users/jingkun/Dropbox/DG/id-list-generator 目录下面
文件名为 09-13-2016_usatocn2013_YU_身份证.pdf

PS:
请把缺失的身份证图片添加到/Users/jingkun/Dropbox/DG/id-list-generator/photo-id 目录下面
如果正反面在同一张图片上，命名为 姓名.jpg
如果正反面分两张图片，命名为 姓名1.jpg 和 姓名2.jpg
